In [1]:
import pandas as pd
import pickle
import numpy as np
import tqdm

get_all_games = 0
get_features = 0
start_val = 500

In [2]:
if get_all_games:
    query = """
    select distinct game_ref_id, date from (
    select distinct game_ref_id, updated_timestamp, dt as date
    from `analytics-156605.rush_app_bi.transactions_ledger_extended_full`
    where transaction_sub_category = 'TotalFee' and transaction_category = 'GamePlay' and platform = 'cashApp' and transaction_status = 'SUCCESS' and dt between "2022-09-01" and "2022-09-15" and game_ref_id like 'LUDO%'
    order by updated_timestamp)
    """

    all_games = pd.read_gbq(query, use_bqstorage_api=True, progress_bar_type='tqdm_notebook')

    all_games = all_games.to_dict('records')

    with open('../data/all_games_sep_1_to_sep_15.pkl', 'wb') as f:
        pickle.dump(all_games, f)

else:

    with open('../data/all_games_sep_1_to_sep_15.pkl', 'rb') as f:
        all_games = pickle.load(f)

In [ ]:
if get_features:
    query = """
    select *
    from `analytics-156605.barath.ludo_prediction_features`
    """

    other_features = pd.read_gbq(query, use_bqstorage_api=True, progress_bar_type='tqdm_notebook')

    other_features = other_features.groupby('game_ref_id').apply(lambda g: g.to_dict('records')).to_dict()

    for name in tqdm.tqdm(other_features):
        other_features[name] = {item['user_id']:item for item in other_features[name]}

    with open('../data/other_features_sep_1_to_sep_15.pkl', 'wb') as f:
        pickle.dump(other_features, f)

else:

    with open('../data/other_features_sep_1_to_sep_15.pkl', 'rb') as f:
        all_games = pickle.load(f)

In [3]:
with open('../data/q_val_data_sep_1_to_sep_5.pkl', 'rb') as f:
    results, results1 = pickle.load(f)

for i in range(len(results)):
    results[i].update(results1[i])

print('1 done')

with open('../data/q_val_data_sep_6_to_sep_10.pkl', 'rb') as f:
    results1, results2 = pickle.load(f)

for i in range(len(results1)):
    results1[i].update(results2[i])

print('2 done')

with open('../data/q_val_data_sep_11_to_sep_15.pkl', 'rb') as f:
    results2, results3 = pickle.load(f)

for i in range(len(results2)):
    results2[i].update(results3[i])

print('3 done')

results = results + results1
del results1
results = results + results2
del results2

data = {item['division']:item for item in results if item is not None}
del results

1 done
2 done
3 done


In [4]:
for dat in tqdm.tqdm(data):
    temp1 = data[dat]['q_user_0'] #== data[dat]['q_max_user_0']
    temp2 = data[dat]['q_user_1'] #== data[dat]['q_max_user_1']
    # temp1 = (data[dat]['q_user_0'] - data[dat]['q_min_user_0']) / (data[dat]['q_max_user_0'] - data[dat]['q_min_user_0'])
    # temp2 = (data[dat]['q_user_1'] - data[dat]['q_min_user_1']) / (data[dat]['q_max_user_1'] - data[dat]['q_min_user_1'])
    # temp1 = temp1.astype('int32')
    # temp2 = temp2.astype('int32')
    if len(temp1) >= 25 and len(temp2) >= 25:
        if len(temp1) != len(temp2):
            length = np.min([len(temp1), len(temp2)])
            temp1 = temp1[:length]
            temp2 = temp2[:length]
        data[dat]['delta_q_0'] = np.mean(temp1 - temp2)
        data[dat]['delta_q_1'] = np.mean(temp2 - temp1)

100%|██████████| 6978165/6978165 [03:59<00:00, 29174.38it/s]


In [5]:
live_scores = {}
live_scores1 = {}

for d in tqdm.tqdm(all_games):
    if d['game_ref_id'] in data and 'delta_q_1' in data[d['game_ref_id']]:
        dat = data[d['game_ref_id']]
        win = dat['score'][0] > dat['score'][1]
        pred = dat['delta_q_0'] > dat['delta_q_1']

        if dat['user_id_0'] not in live_scores:
            live_scores[dat['user_id_0']] = start_val
        if dat['user_id_1'] not in live_scores:
            live_scores[dat['user_id_1']] = start_val
        if dat['user_id_0'] not in live_scores1:
            live_scores1[dat['user_id_0']] = start_val
        if dat['user_id_1'] not in live_scores1:
            live_scores1[dat['user_id_1']] = start_val

        data[d['game_ref_id']]['q_score_0'] = live_scores[dat['user_id_0']]
        data[d['game_ref_id']]['q_score_1'] = live_scores[dat['user_id_1']]
        data[d['game_ref_id']]['score_score_0'] = live_scores1[dat['user_id_0']]
        data[d['game_ref_id']]['score_score_1'] = live_scores1[dat['user_id_1']]

        data[d['game_ref_id']]['dt'] = str(d['date'])

        data[d['game_ref_id']]['hour'] = other_features[d['game_ref_id']]['user_id_0']['hour']

        data[d['game_ref_id']]['table_0'] = other_features[d['game_ref_id']]['user_id_0']['table']
        data[d['game_ref_id']]['table_1'] = other_features[d['game_ref_id']]['user_id_1']['table']

        data[d['game_ref_id']]['wallet_0'] = other_features[d['game_ref_id']]['user_id_0']['wallet']
        data[d['game_ref_id']]['wallet_1'] = other_features[d['game_ref_id']]['user_id_1']['wallet']

        if win == pred:
            live_scores[dat['user_id_0']] += dat['delta_q_0']
            live_scores[dat['user_id_1']] += dat['delta_q_1']

        live_scores1[dat['user_id_0']] += (dat['score'][0] - dat['score'][1])
        live_scores1[dat['user_id_1']] += (dat['score'][1] - dat['score'][0])

100%|██████████| 9598580/9598580 [01:45<00:00, 90879.24it/s]


In [104]:
win = []
q_pred = []
skill_pred = []
q_score_0 = []
q_score_1 = []

feats = []
feats_test = []
y = []
y_test = []

which_feats = 3 # 0 - skill, 1 - game + skill, 2 - q + skill, 3 - q + game + skill, 4 - 3 + other_feats

for dat in tqdm.tqdm(data):
    if 'delta_q_1' in data[dat] and 'dt' in data[dat]:
        if data[dat]['dt'] in ['2022-09-11', '2022-09-12', '2022-09-13', '2022-09-14', '2022-09-15']:
            win.append(data[dat]['score'][0] > data[dat]['score'][1])
            q_pred.append(data[dat]['q_score_0'] > data[dat]['q_score_1'])
            skill_pred.append((data[dat]['mu_0'] - 3*data[dat]['sigma_0']) > (data[dat]['mu_1'] - 3*data[dat]['sigma_1']))

            q_score_0.append(data[dat]['score_score_0'])
            q_score_1.append(data[dat]['score_score_1'])

        if data[dat]['dt'] in ['2022-09-11', '2022-09-12', '2022-09-13']:

            if which_feats == 0:
                feats.append([data[dat]['mu_0'], data[dat]['sigma_0'], 
                            data[dat]['mu_1'], data[dat]['sigma_1']])
            elif which_feats == 1:
                feats.append([data[dat]['mu_0'], data[dat]['sigma_0'], data[dat]['score_score_0'], 
                            data[dat]['mu_1'], data[dat]['sigma_1'], data[dat]['score_score_1']])
            elif which_feats == 2:
                feats.append([data[dat]['mu_0'], data[dat]['sigma_0'], data[dat]['q_score_0'], 
                            data[dat]['mu_1'], data[dat]['sigma_1'], data[dat]['q_score_1']])
            elif which_feats == 3:
                feats.append([data[dat]['mu_0'], data[dat]['sigma_0'], data[dat]['q_score_0'], data[dat]['score_score_0'], 
                            data[dat]['mu_1'], data[dat]['sigma_1'], data[dat]['q_score_1'], data[dat]['score_score_1']])
            elif which_feats == 4:
                feats.append([data[dat]['mu_0'], data[dat]['sigma_0'], data[dat]['q_score_0'], data[dat]['score_score_0'], 
                            data[dat]['mu_1'], data[dat]['sigma_1'], data[dat]['q_score_1'], data[dat]['score_score_1'],
                            data[dat]['table_0'], data[dat]['table_1'], data['wallet_0'], data['wallet_1'], data['hour']])

            y.append(data[dat]['score'][0] > data[dat]['score'][1])

        elif data[dat]['dt'] in ['2022-09-14', '2022-09-15']:

            if which_feats == 0:
                feats_test.append([data[dat]['mu_0'], data[dat]['sigma_0'], 
                            data[dat]['mu_1'], data[dat]['sigma_1']])
            elif which_feats == 1:
                feats_test.append([data[dat]['mu_0'], data[dat]['sigma_0'], data[dat]['score_score_0'], 
                            data[dat]['mu_1'], data[dat]['sigma_1'], data[dat]['score_score_1']])
            elif which_feats == 2:
                feats_test.append([data[dat]['mu_0'], data[dat]['sigma_0'], data[dat]['q_score_0'], 
                            data[dat]['mu_1'], data[dat]['sigma_1'], data[dat]['q_score_1']])
            elif which_feats == 3:
                feats_test.append([data[dat]['mu_0'], data[dat]['sigma_0'], data[dat]['q_score_0'], data[dat]['score_score_0'], 
                            data[dat]['mu_1'], data[dat]['sigma_1'], data[dat]['q_score_1'], data[dat]['score_score_1']])
            elif which_feats == 4:
                feats_test.append([data[dat]['mu_0'], data[dat]['sigma_0'], data[dat]['q_score_0'], data[dat]['score_score_0'], 
                            data[dat]['mu_1'], data[dat]['sigma_1'], data[dat]['q_score_1'], data[dat]['score_score_1'],
                            data[dat]['table_0'], data[dat]['table_1'], data['wallet_0'], data['wallet_1'], data['hour']])

            y_test.append(data[dat]['score'][0] > data[dat]['score'][1])

100%|██████████| 6978165/6978165 [00:38<00:00, 179993.70it/s]


In [105]:
win = np.array(win).astype('int32')
q_pred = np.array(q_pred).astype('int32')
skill_pred = np.array(skill_pred).astype('int32')

feats = np.array(feats)
feats_test = np.array(feats_test)
y = np.array(y)
y_test = np.array(y_test)

In [106]:
np.sum(win == skill_pred)/len(win)

0.4771809054127549

In [107]:
np.sum(win == q_pred)*100/len(win)

51.166519466328594

In [108]:
q_score_0 = np.array(q_score_0)
q_score_1 = np.array(q_score_1)

bins = np.abs(q_score_0 - q_score_1)

In [109]:
u1 = []
tot = []
for i in range(25):
    idx = np.logical_and(bins>i*200, bins<(i+1)*200)
    temp = win[idx]
    u1.append(np.sum(temp[q_score_0[idx]>q_score_1[idx]]) + np.sum(temp[q_score_0[idx]<q_score_1[idx]]==0))
    tot.append(len(q_score_0[idx]))

In [110]:
import pandas as pd
pd.DataFrame(np.array([u1, tot]).T, columns=['1', '2'])

,1,2
0,194047,387151
1,163453,324092
2,131628,259536
3,106062,208401
4,86796,169450
5,70797,138252
6,59484,114499
7,49549,95359
8,41709,79738
9,35156,66884


In [111]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier

In [112]:
clf = LGBMClassifier(random_state=42, verbose=1)
# clf = Pipeline([('scaler', StandardScaler()), ('gbc', GradientBoostingClassifier(random_state=42, verbose=1))])
# clf = Pipeline([('scaler', StandardScaler()), ('gbc', LogisticRegression(random_state=42, verbose=1))])
clf = clf.fit(feats, y)

[LightGBM] [Info] Number of positive: 790042, number of negative: 658356
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.195954 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2040
[LightGBM] [Info] Number of data points in the train set: 1448398, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.545459 -> initscore=0.182340
[LightGBM] [Info] Start training from score 0.182340


In [113]:
y_pred = clf.predict(feats_test)
y_probs = clf.predict_proba(feats_test)

In [114]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, ConfusionMatrixDisplay

def perf(ytrue, ypred):
    accu = accuracy_score(ytrue, ypred)
    prec = precision_score(ytrue, ypred)
    rec = recall_score(ytrue, ypred)
    return accu*100, prec*100, rec*100

In [115]:
acc, precision, recall = perf(y_test, y_pred)
print(acc, precision, recall)

55.572473702098115 56.036342903205195 86.74527826336723


In [116]:
bins = np.abs(y_probs[:,1]-0.5)

In [117]:
u1 = []
tot = []
for i in range(25):
    idx = np.logical_and(bins>i*0.05, bins<(i+1)*0.05)
    temp = y_test[idx]
    u1.append(np.sum(temp[y_probs[:,1][idx]>=0.5]==1) + np.sum(temp[y_probs[:,1][idx]<0.5]==0))
    tot.append(len(y_test[idx]))

In [118]:
pd.DataFrame(np.array([u1, tot]).T, columns=['1', '2'])

,1,2
0,271734,514722
1,178890,314184
2,61230,99351
3,19353,29029
4,3372,4723
5,110,139
6,1,1
7,0,0
8,0,0
9,0,0
